# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [ ]:
!pip install pymorphy2
from sklearn.feature_extraction.text import CountVectorizer
import pymorphy2
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.metrics.distance import edit_distance
import random
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial import distance

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. 

In [ ]:
text = '''с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий'''

In [ ]:
with open('litw-win.txt', encoding = 'cp1251') as f:
    words1 = [row.split()[-1] for row in f]
text_words = list(text.split())

for i in range(len(text_words)):
    if text_words[i] not in words1:
        distances = {j : edit_distance(text_words[i], j) for j in words1}
        sorted_distances = sorted(distances.items(), key = lambda x: x[1])
        text_words[i] = sorted_distances[0][0]
        
print(' '.join(text_words))

с величайшим усилием выбравшись из потока убегающих людей кутузов со свитой уменьшившейся вдвое поехал на звуки выстрелов русских орудий


2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.

In [ ]:
text2 = 'Считайте слова из файла litw-win.txt и запишите их в список words. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка words. Считайте, что в слове есть опечатка, если данное слово не содержится в списке words'

In [ ]:
stemmer = SnowballStemmer('russian')
morph = pymorphy2.MorphAnalyzer()
text2_words = [word for word in word_tokenize(str(text2)) if word.isalpha()]
res1,res2='',''
for i in text2_words:
    res1+=stemmer.stem(i)+' '
for i in text2_words:
    res2+=morph.parse(i)[0].normalized.word+' '
print(f'Стемминг: {res1}\n\nЛемматизация: {res2}')

Стемминг: счита слов из файл и запиш их в список words в зада предложен исправьт все опечатк замен слов с опечатк на ближайш в смысл расстоян левенштейн к ним слов из списк words счита что в слов ест опечатк есл дан слов не содерж в списк words 

Лемматизация: считать слово из файл и записать они в список words в задать предложение исправить всё опечатка заменить слово с опечатка на близкий в смысл расстояние левенштейн к они слово из список words считать что в слово есть опечатка если данный слово не содержаться в список words 


3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи `CountVectorizer`.

In [ ]:
CountVectorizer().fit_transform(sent_tokenize(text2)).toarray()

array([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
        1, 1, 2, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
        0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1]], dtype=int64)

## Лабораторная работа 9

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [ ]:
data = pd.read_csv('preprocessed_descriptions.csv')
data

,Unnamed: 0,name,preprocessed_descriptions
0,0,george s at the cove black bean soup,an original recipe created by chef scott meska...
1,1,healthy for them yogurt popsicles,my children and their friends ask for my homem...
2,2,i can t believe it s spinach,these were so go it surprised even me
3,3,italian gut busters,my sisterinlaw made these for us at a family g...
4,4,love is in the air beef fondue sauces,i think a fondue is a very romantic casual din...
...,...,...,...
29995,29995,zurie s holey rustic olive and cheddar bread,this is based on a french recipe but i changed...
29996,29996,zwetschgenkuchen bavarian plum cake,this is a traditional fresh plum cake thought ...
29997,29997,zwiebelkuchen southwest german onion cake,this is a traditional late summer early fall s...
29998,29998,zydeco soup,this is a delicious soup that i originally fou...


In [ ]:
words = []
for i in data['preprocessed_descriptions']:
    words.extend([word for word in word_tokenize(str(i)) if word.isalpha()])
words = set(words)
print(words)

{'bouillabaisse', 'orange', 'jerry', 'dennis', 'skahill', 'mandoline', 'coma', 'switzerland', 'likeim', 'hassan', 'angies', 'official', 'rectangles', 'lodge', 'wwwgentlethanksgivingorg', 'morningstar', 'shoulder', 'cellars', 'daytons', 'preferfrom', 'havenote', 'alas', 'bury', 'desires', 'oilit', 'foccaccia', 'rhodes', 'christams', 'twohandled', 'awayand', 'soupsthese', 'clafouti', 'legally', 'thymeflecked', 'folding', 'sweets', 'sweetsour', 'inventing', 'sandwichpanini', 'cringed', 'gooseberry', 'cult', 'spa', 'pepperoncini', 'lightprepare', 'augsburg', 'carbonada', 'mandarinstyle', 'montbazillac', 'coat', 'tabascos', 'jaffreys', 'adult', 'acknowledge', 'ery', 'soupy', 'yal', 'soup', 'googled', 'purposely', 'formerly', 'mantou', 'tohi', 'quadra', 'steamable', 'desired', 'names', 'jackolantern', 'j', 'picnicbbq', 'pilsubury', 'straightup', 'rosenbergs', 'corwinn', 'caketype', 'pomegranatelime', 'slant', 'sweetens', 'arab', 'mommom', 'trues', 'crusthard', 'monkeyed', 'hispanic', 'forehe

In [ ]:
print(f'Уникальных слов: {len(words)}')

Уникальных слов: 30743


1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [ ]:
for i in range(5):
    w1 = random.sample(list(words), 1)[0]
    w2 = random.sample(list(words), 1)[0]
    print(f'Слова: {w1}, {w2}, расстояние редактирования: {edit_distance(w1, w2)}')

Слова: celiyak, leyte, расстояние редактирования: 5
Слова: ambya, kneppi, расстояние редактирования: 6
Слова: ingrid, savored, расстояние редактирования: 5
Слова: setplease, bridges, расстояние редактирования: 7
Слова: bordeaux, store, расстояние редактирования: 6


1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [ ]:
def nearest(word, k):
    dists = {i : edit_distance(word, i) for i in words}
    sorted_dists = sorted(dists.items(), key = lambda x: x[1])
    return sorted_dists[:k]

In [ ]:
nearest('details', 5)

[('details', 0), ('detail', 1), ('tails', 2), ('devils', 2), ('retains', 2)]

### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [ ]:
stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()
df_words = pd.DataFrame(words, columns = ['word'])
df_words['stemmed_word'] = df_words.apply(lambda x: stemmer.stem(x['word']), axis = 1)
df_words['normalized_word'] = df_words.apply(lambda x: lemmatizer.lemmatize(x['word']), axis = 1)
df_words = df_words.set_index('word')
df_words

,stemmed_word,normalized_word
word,,
bouillabaisse,bouillabaiss,bouillabaisse
orange,orang,orange
jerry,jerri,jerry
dennis,denni,dennis
skahill,skahil,skahill
...,...,...
expert,expert,expert
allout,allout,allout
httpwwwfoodcomrecipefindercondimentsetcraselhanout,httpwwwfoodcomrecipefindercondimentsetcraselha...,httpwwwfoodcomrecipefindercondimentsetcraselha...


2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [ ]:
stop_words = stopwords.words('english')
words_no_stop = [word for word in all_words if word not in stop_words]
print(f'Доля стоп слов составляла: {(len(all_words) - len(words_no_stop)) / len(all_words) * 100}')

Доля стоп слов составляла: 46.10722870409009


In [ ]:
for k, v in nltk.FreqDist(all_words).most_common(10):
    print(f'{k} - {v}')

the - 40072
a - 34951
and - 30245
this - 26859
i - 24836
to - 23471
is - 20285
it - 19756
of - 18364
for - 15939


In [ ]:
for k, v in nltk.FreqDist(words_no_stop).most_common(10):
    print(f'{k} - {v}')

recipe - 14871
make - 6326
time - 5137
use - 4620
great - 4430
like - 4167
easy - 4152
one - 3872
made - 3810
good - 3791


### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

In [ ]:
data3 = data.sample(5)
data3

,Unnamed: 0,name,preprocessed_descriptions
8311,8311,creamy orange dressing raw vegan,ive been trying to come up with different ways...
7638,7638,corned beef in beer crock pot,the best corned beef ive ever made was in the ...
2488,2488,bbq green beans,these are reallllllly good
29703,29703,yorkshire spice bread for the a b m,this is from the book giant book of bread mach...
13217,13217,ham and cheese croissants just like sara le...,these are ham crescent rolls with a melted cre...


In [ ]:
tv = TfidfVectorizer()
for i in data3['preprocessed_descriptions']:
    print(tv.fit_transform(sent_tokenize(i)).toarray(), '\n')

[[0.12751534 0.06375767 0.06375767 0.12751534 0.06375767 0.06375767
  0.06375767 0.12751534 0.06375767 0.06375767 0.06375767 0.06375767
  0.06375767 0.06375767 0.06375767 0.06375767 0.12751534 0.06375767
  0.06375767 0.12751534 0.06375767 0.06375767 0.06375767 0.06375767
  0.06375767 0.06375767 0.06375767 0.06375767 0.06375767 0.06375767
  0.06375767 0.06375767 0.19127301 0.06375767 0.06375767 0.06375767
  0.12751534 0.06375767 0.38254603 0.06375767 0.19127301 0.06375767
  0.19127301 0.06375767 0.06375767 0.19127301 0.06375767 0.06375767
  0.06375767 0.06375767 0.06375767 0.06375767 0.06375767 0.06375767
  0.06375767 0.06375767 0.12751534 0.25503069 0.06375767 0.06375767
  0.06375767 0.06375767 0.06375767 0.06375767 0.06375767 0.06375767
  0.06375767 0.06375767 0.06375767 0.06375767 0.06375767 0.06375767
  0.06375767 0.06375767 0.06375767 0.19127301 0.12751534 0.12751534
  0.06375767 0.19127301 0.06375767 0.06375767 0.06375767 0.06375767
  0.06375767 0.06375767 0.06375767 0.06375767 0.

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

In [ ]:
from scipy.spatial.distance import cosine
data3['preprocessed_descriptions'] = data3['preprocessed_descriptions'].astype(str)
tv = TfidfVectorizer()
vectors = tv.fit_transform(data3['preprocessed_descriptions']).toarray()
df3_2 = pd.DataFrame(index = data3['name'], columns = data3['name'])
for i, r1 in enumerate(data3['name']):
    for j, r2 in enumerate(data3['name']):
        df3_2.at[r1, r2] = 1 - cosine(vectors[i], vectors[j])
df3_2

name,creamy orange dressing raw vegan,corned beef in beer crock pot,bbq green beans,yorkshire spice bread for the a b m,ham and cheese croissants just like sara lee copycat
name,,,,,
creamy orange dressing raw vegan,1,0.124418,0.03626,0.181885,0.176622
corned beef in beer crock pot,0.124418,1,0.0,0.151444,0.194009
bbq green beans,0.03626,0.0,1,0.0,0.141152
yorkshire spice bread for the a b m,0.181885,0.151444,0.0,1,0.298595
ham and cheese croissants just like sara lee copycat,0.176622,0.194009,0.141152,0.298595,1


3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).

In [ ]:
res = df3_2.where(df3_2 != 1).max().max()
res

0.29859480485508083

наиболее похожие рецепты: "yorkshire spice bread for the a b m" и "ham and cheese croissants just like sara lee copycat"